In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time

In [2]:
chrome_options = webdriver.ChromeOptions()

In [3]:
chrome_options.binary_location = 'C:/chromedriver/chrome-win64/chrome.exe'

In [4]:
service = Service('C:/chromedriver/chromedriver-win64/chromedriver.exe')

In [5]:
chrome_options.add_argument('--start-maximized')

In [6]:
url = 'https://www.google.com/'

In [7]:
# initialize the driver
driver = webdriver.Chrome(service=service,options=chrome_options)

try:
    driver.get(url)

    search_input_element = driver.find_element(By.NAME, 'q')
    
    search_input_element.send_keys('sauce demo')
    driver.implicitly_wait(1)
    
    search_input_element.send_keys(Keys.RETURN)

    stop = True
    while stop:
        driver.implicitly_wait(2)
               
        #find a particular site
        search_result_top = driver.find_element(By.ID, 'rso')
        driver.implicitly_wait(1)
    
        search_results_in = search_result_top.find_elements(By.CLASS_NAME, 'yuRUbf')
        
        # print(len(search_results_in))
        
        for result in search_results_in:
            domain_title = result.find_element(By.CLASS_NAME, 'VuuXrf')
            title = domain_title.text
            
            if 'Swag Labs'.lower() in title.strip().lower():  
                href = result.find_element(By.TAG_NAME,'a')
                new_link = href.get_attribute('href').split(".com")[0] + ".com"
                
                stop = False
                break
        if not stop:
            break
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(1)  
        
        next = driver.find_element(By.CLASS_NAME, 'oeN89d')  
        next.click()        
        
    # take saucedemo link and place order
    try:
        driver.get(new_link)
        driver.implicitly_wait(2)
        title = driver.title
        if title == 'Swag Labs':
            try:
                time.sleep(1)
                username = driver.find_element(By.ID, 'user-name')
                username.send_keys('standard_user')
                time.sleep(1)
                password = driver.find_element(By.ID, 'password')
                password.send_keys('secret_sauce')
                time.sleep(1)
                login = driver.find_element(By.ID, 'login-button')
                time.sleep(2)
                login.click()

                # verify login and add 3 items to cart
                verify = driver.find_element(By.CLASS_NAME, "app_logo").text
                if verify == 'Swag Labs':
                    add_backpack = driver.find_element(By.ID, 'add-to-cart-sauce-labs-backpack')
                    time.sleep(1)
                    add_backpack.click()
                    add_shirt = driver.find_element(By.ID, 'add-to-cart-sauce-labs-bolt-t-shirt')
                    time.sleep(1)
                    add_shirt.click()                    
                    add_sweat = driver.find_element(By.ID, 'add-to-cart-test.allthethings()-t-shirt-(red)')
                    time.sleep(1)
                    add_sweat.click()

                    time.sleep(1)
                    # click on cart
                    cart = driver.find_element(By.CLASS_NAME, 'shopping_cart_link')
                    cart.click()

                    list_of_items = []
                    prices = []

                    checkout_list = []
                    checkout_prices = []
                    
                    driver.implicitly_wait(2)
                    #verify 3 items in cart
                    unique_items_in_cart = driver.find_element(By.CLASS_NAME, 'cart_list')
                    count = unique_items_in_cart.find_elements(By.CLASS_NAME, 'cart_item')
                    if len(count) == 3:
                        for item in count:
                            item_name = item.find_element(By.CLASS_NAME, 'inventory_item_name').text
                            list_of_items.append(item_name)
                            price = item.find_element(By.CLASS_NAME, 'inventory_item_price').text
                            prices.append(float(price.replace('$', '')))
                        # chekout 
                        time.sleep(2)
                        checkout = driver.find_element(By.ID, 'checkout')
                        checkout.click()

                        # fill checkout details
                        fname = driver.find_element(By.ID, 'first-name')
                        time.sleep(1)
                        fname.send_keys('Philip')
                        lname = driver.find_element(By.ID, 'last-name')
                        time.sleep(1)
                        lname.send_keys('Alfred')
                        pcode = driver.find_element(By.ID, 'postal-code')
                        time.sleep(1)
                        pcode.send_keys('930001')
                        
                        time.sleep(2)
                        proceed = driver.find_element(By.ID, 'continue')
                        proceed.click()

                    # checkout overview
                    overview_list = driver.find_element(By.CLASS_NAME, 'cart_list')
                    overview_item = overview_list.find_elements(By.CLASS_NAME, 'cart_item')
                    for item in overview_item:
                        name = item.find_element(By.CLASS_NAME, 'inventory_item_name').text
                        checkout_list.append(name)
                        price = item.find_element(By.CLASS_NAME, 'inventory_item_price').text
                        checkout_prices.append(float(price.replace('$', '')))

                    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    
                    assert sorted(list_of_items) == sorted(checkout_list), 'Items in cart and checkout do not match'
                    assert sum(prices) == sum(checkout_prices), 'Sum total in Cart and Checkout do not match'
                    finish = driver.find_element(By.ID, 'finish')
                    time.sleep(2)
                    finish.click()

                    # Thank you for your order! ahould be visble after clicking finish
                    order_complete = driver.find_element(By.CLASS_NAME, 'complete-header').text
                    if order_complete == 'Thank you for your order!':
                        print('Order Placed Successfully!')
            except Exception as e:
                print(f'{e}')
                
        assert title == "Swag Labs", "Page title does not match"       
        time.sleep(2)
    except Exception as e:
        print(f'{e}')
except Exception as e:
    print(f'{e}')
finally:
    driver.quit()


Order Placed Successfully!
